In [16]:
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
import pytorch3d
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    TexturesUV,
    TexturesVertex
)
import numpy as np 

In [17]:
cameras = pytorch3d.renderer.FoVPerspectiveCameras(
    R=torch.eye(3).unsqueeze(0),
    T=torch.tensor([[0, 0, 3]]),
    fov=60,
)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:

# lights = PointLights(device=device, location=[[0.0, 0.0, 3.0]])
from pytorch3d.renderer import PointLights

# Key Light: Front-right, slightly above
key_light = [3.0, 3.0, 2.0]

# Fill Light: Front-left, lower intensity
fill_light = [-3.0, 2.0, 2.0]

# Back Light: Behind and slightly above the object
back_light = [0.0, -3.0, 3.0]

# Define the light colors (Key light is stronger, Fill light weaker)
# Each color is in the form [R, G, B] with values between 0 and 1.
ambient_color = [[0.1, 0.1, 0.1]] * 3   # Ambient light for all lights
diffuse_color = [[1.0, 1.0, 1.0],       # Key light diffuse color
                 [0.5, 0.5, 0.5],       # Fill light diffuse color
                 [0.8, 0.8, 0.8]]       # Back light diffuse color
specular_color = [[0.5, 0.5, 0.5]] * 3  # Specular highlights for all lights

# Create the three-point lighting setup
lights = PointLights(
    location=[key_light, fill_light, back_light],
    ambient_color=ambient_color,
    diffuse_color=diffuse_color,
    specular_color=specular_color
)
# Set up the renderer
raster_settings = RasterizationSettings(
    image_size=512, 
    blur_radius=0.0, 
    faces_per_pixel=1, 
)
renderer = MeshRenderer(
    rasterizer=MeshRasterizer(
        cameras=cameras, 
        raster_settings=raster_settings
    ),
    shader=SoftPhongShader(
        device=device, 
        cameras=cameras,
        lights=lights
    )
)

In [26]:
import requests

# URL of the Python script 
url = "https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py"

# Send a GET request to the URL
response = requests.get(url)

# Save the content of the response as a Python file
with open("plot_image_grid.py", "wb") as file:
    file.write(response.content)

from plot_image_grid import image_grid

In [27]:
# Setup
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

# Set paths
DATA_DIR = "./data"
obj_filename = 'model.obj'


# Load obj file
mesh = load_objs_as_meshes([obj_filename], device=device)

In [28]:
# Extract vertices, faces, and textures
vertices = mesh.verts_packed()  # Shape: (N_v, 3)
faces = mesh.faces_packed()      # Shape: (N_f, 3)
# Extract textures, if available
if mesh.textures is not None:
    textures = mesh.textures.verts_rgb_packed()  # Shape: (N_v, 3)
else:
    # If no textures are available, you can create a dummy texture
    textures = torch.ones_like(vertices)  # Create a white color for each vertex
vertices = vertices.unsqueeze(0)  
faces = faces.unsqueeze(0)          
textures = textures.unsqueeze(0)    
# Print the shapes of the extracted components
print("Vertices shape:", vertices.shape)  
print("Faces shape:", faces.shape)          
print("Textures shape:", textures.shape)    
# Add a batch dimension


Vertices shape: torch.Size([1, 377, 3])
Faces shape: torch.Size([1, 714, 3])
Textures shape: torch.Size([1, 377, 3])


In [29]:
meshes = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(textures),
)

In [31]:
import imageio
import numpy as np
num_frames = 60  # Number of frames for the GIF
angles = torch.linspace(0, 360, num_frames)  # Angles for full rotation
images = []  # To store the rendered images

for angle in angles:
    # Compute the camera position and orientation
    R, T = look_at_view_transform(dist=3, elev=5, azim=angle.item())
    cameras = FoVPerspectiveCameras(R=R, T=T, device=device)

    rend = renderer(meshes, cameras=cameras, lights=lights)
    
    # Get the RGB image
    image = rend[0, ..., :3].cpu().numpy()  # Shape: (H, W, 3)
    # Normalize the image to [0, 1] and convert to uint8
    # Check for non-zero values before normalization
    if image.max() > 0:  # Only normalize if there's a non-zero value
        epsilon = 1e-8
        image = (image - image.min()) / (image.max() - image.min() + epsilon)
    
    # Convert to uint8
    image = (image * 255).astype(np.uint8)  # Convert to uint8
    
    images.append(image)

# Save the images as a GIF
imageio.mimsave('Pikachu_360_degree_render3.gif', images, fps=15)

# Optional: Display the first image
plt.figure(figsize=(8, 8))
plt.imshow(images[0])  # Display the first frame
plt.axis("off")
plt.show()

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2473901162496 bytes.